In [12]:
import numpy as np
import mne
import matplotlib.pyplot as plt
import pandas as pd
from mne.preprocessing import ICA
from tqdm.notebook import tqdm

In [10]:
!mkdir data
!mkdir data/ME
!mkdir data/MI

In [13]:
def convertToNumpy(raws):
    raw_data = mne.concatenate_raws(raws)
    event_id = {'1536': 1, '1537': 2, '1538': 3, '1539': 4, '1540': 5, '1541': 6, '1542': 7}
    events_A, event_id_A = mne.events_from_annotations(raw_data, event_id=event_id)

    raw_data.info['bads'] += ['thumb_near', 'thumb_far', 'thumb_index', 'index_near', 'index_far', 'index_middle', 'middle_near', 'middle_far', 'middle_ring', 'ring_near', 'ring_far', 'ring_little', 'litte_near', 'litte_far', 'thumb_palm', 'wrist_bend', 'roll', 'pitch', 'gesture', 'handPosX', 'handPosY', 'handPosZ', 'elbowPosX', 'elbowPosY', 'elbowPosZ', 'ShoulderAdductio', 'ShoulderFlexionE', 'ShoulderRotation', 'Elbow', 'ProSupination', 'Wrist', 'GripPressure']
    picks = mne.pick_types(raw_data.info, meg=False, eeg=True, eog=True, stim=False, exclude='bads')
    event_id={'1':1, '2': 2, '3': 3, '4':4,'5':5,'6':6,'7':7 }
    epochs = mne.Epochs(raw_data, events_A, event_id, tmin=-0.01, tmax=5.0, proj=True, picks=picks, preload=True)


    print("*************************************************************")
    print("Applying butterworth filter")
    print("*************************************************************")
    iir_params = dict(order=4, ftype='butter')
    epochs.filter(l_freq=0.5, h_freq=70.0, method='iir', iir_params=iir_params, verbose=True)

    print("*************************************************************")
    print("Applying ICA")
    print("*************************************************************")
    ica = ICA(n_components=15, random_state=97, max_iter=800)
    ica.fit(epochs)
    ica.exclude = [0]
    epochs = ica.apply(epochs)

    print("*************************************************************")
    print("Applying baseline correction")
    print("*************************************************************")
    epochs.apply_baseline(baseline=(None, 0))

    labels = epochs.events[:,-1]
    data = epochs.get_data()
    return data,labels

In [26]:
for i in tqdm(range(2,16)):
    stringName = f"S0{i}" if i < 10 else f"S{i}"
    
    ## For ME
    !mkdir data/ME/{stringName}
    raws = []
    for j in range(1,11):
        raws.append(mne.io.read_raw_gdf(f'/kaggle/input/bnci2020/{stringName}_ME/motorexecution_subject{i}_run{j}.gdf', preload=True))
        
    data, labels = convertToNumpy(raws)
    np.save(f"data/ME/{stringName}/data.npy", data)
    np.save(f"data/ME/{stringName}/labels.npy", labels)
    
    
    ## For MI
    !mkdir data/MI/{stringName}
    raws = []
    for j in range(1,11):
        raws.append(mne.io.read_raw_gdf(f'/kaggle/input/bnci2020/{stringName}_MI/motorimagination_subject{i}_run{j}.gdf', preload=True))
        
    data, labels = convertToNumpy(raws)
    np.save(f"data/MI/{stringName}/data.npy", data)
    np.save(f"data/MI/{stringName}/labels.npy", labels)
    

  0%|          | 0/14 [00:00<?, ?it/s]

Extracting EDF parameters from /kaggle/input/bnci2020/S02_ME/motorexecution_subject2_run1.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
F3, F1, Fz, F2, F4, FFC5h, FFC3h, FFC1h, FFC2h, FFC4h, FFC6h, FC5, FC3, FC1, FCz, FC2, FC4, FC6, FTT7h, FCC5h, FCC3h, FCC1h, FCC2h, FCC4h, FCC6h, FTT8h, C5, C3, C1, Cz, C2, C4, C6, TTP7h, CCP5h, CCP3h, CCP1h, CCP2h, CCP4h, CCP6h, TTP8h, CP5, CP3, CP1, CPz, CP2, CP4, CP6, CPP5h, CPP3h, CPP1h, CPP2h, CPP4h, CPP6h, P3, P1, Pz, P2, P4, PPO1h, PPO2h, eog-l, eog-m, eog-r, thumb_near, thumb_far, thumb_index, index_near, index_far, index_middle, middle_near, middle_far, middle_ring, ring_near, ring_far, ring_little, litte_near, litte_far, thumb_palm, wrist_bend, roll, pitch, gesture, handPosX, handPosY, handPosZ, elbowPosX, elbowPosY, elbowPosZ, ShoulderAdductio, ShoulderFlexionE, ShoulderRotation, Elbow, ProSupination, Wrist, GripPressure
Creating raw.info struct

/tmp/ipykernel_30/521598588.py:22: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs)


Selecting by number: 15 components
Fitting ICA took 40.2s.
Applying ICA to Epochs instance


/tmp/ipykernel_30/521598588.py:24: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  epochs = ica.apply(epochs)


...

    Transforming to ICA space (15 components)
    Zeroing out 1 ICA component
    Projecting back using 64 PCA components
*************************************************************
Applying baseline correction
*************************************************************
Applying baseline correction (mode: mean)
Extracting EDF parameters from /kaggle/input/bnci2020/S15_MI/motorimagination_subject15_run1.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
F3, F1, Fz, F2, F4, FFC5h, FFC3h, FFC1h, FFC2h, FFC4h, FFC6h, FC5, FC3, FC1, FCz, FC2, FC4, FC6, FTT7h, FCC5h, FCC3h, FCC1h, FCC2h, FCC4h, FCC6h, FTT8h, C5, C3, C1, Cz, C2, C4, C6, TTP7h, CCP5h, CCP3h, CCP1h, CCP2h, CCP4h, CCP6h, TTP8h, CP5, CP3, CP1, CPz, CP2, CP4, CP6, CPP5h, CPP3h, CPP1h, CPP2h, CPP4h, CPP6h, P3, P1, Pz, P2, P4, PPO1h, PPO2h, eog-l, eog-m, eog-r, thumb_near, thumb_far, thumb_index, index_near, index_far, index_middle, midd

/tmp/ipykernel_30/521598588.py:22: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs)


Selecting by number: 15 components
Fitting ICA took 51.8s.
Applying ICA to Epochs instance


/tmp/ipykernel_30/521598588.py:24: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  epochs = ica.apply(epochs)


    Transforming to ICA space (15 components)
    Zeroing out 1 ICA component
    Projecting back using 64 PCA components
*************************************************************
Applying baseline correction
*************************************************************
Applying baseline correction (mode: mean)


In [31]:
raws = []
for j in range(1,11):
    raws.append(mne.io.read_raw_gdf(f'/kaggle/input/bnci2020/S01_MI/motorimagination_subject1_run{j}.gdf', preload=True))
raw_data = mne.concatenate_raws(raws)
event_id = {'1536': 1, '1537': 2, '1538': 3, '1539': 4, '1540': 5, '1541': 6, '1542': 7}
events_A, event_id_A = mne.events_from_annotations(raw_data, event_id=event_id)

raw_data.info['bads'] += ['thumb_near',
 'thumb_far',
 'thumb_index',
 'index_near',
 'index_far',
 'index_middle',
 'middle_near',
 'middle_far',
 'middle_ring',
 'ring_near',
 'ring_far',
 'ring_little',
 'litte_near',
 'litte_far',
 'thumb_palm',
 'wrist_bend',
 'roll',
 'pitch',
 'gesture',
 'armeodummy-0',
 'armeodummy-1',
 'armeodummy-2',
 'armeodummy-3',
 'armeodummy-4',
 'armeodummy-5',
 'armeodummy-6',
 'armeodummy-7',
 'armeodummy-8',
 'armeodummy-9',
 'armeodummy-10',
 'armeodummy-11',
 'armeodummy-12']
picks = mne.pick_types(raw_data.info, meg=False, eeg=True, eog=True, stim=False, exclude='bads')
event_id={'1':1, '2': 2, '3': 3, '4':4,'5':5,'6':6,'7':7 }
epochs = mne.Epochs(raw_data, events_A, event_id, tmin=-0.01, tmax=5.0, proj=True, picks=picks, preload=True)


print("*************************************************************")
print("Applying butterworth filter")
print("*************************************************************")
iir_params = dict(order=4, ftype='butter')
epochs.filter(l_freq=0.5, h_freq=70.0, method='iir', iir_params=iir_params, verbose=True)

print("*************************************************************")
print("Applying ICA")
print("*************************************************************")
ica = ICA(n_components=15, random_state=97, max_iter=800)
ica.fit(epochs)
ica.exclude = [0]
epochs = ica.apply(epochs)

print("*************************************************************")
print("Applying baseline correction")
print("*************************************************************")
epochs.apply_baseline(baseline=(None, 0))

labels = epochs.events[:,-1]
data = epochs.get_data()

Extracting EDF parameters from /kaggle/input/bnci2020/S01_MI/motorimagination_subject1_run1.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
F3, F1, Fz, F2, F4, FFC5h, FFC3h, FFC1h, FFC2h, FFC4h, FFC6h, FC5, FC3, FC1, FCz, FC2, FC4, FC6, FTT7h, FCC5h, FCC3h, FCC1h, FCC2h, FCC4h, FCC6h, FTT8h, C5, C3, C1, Cz, C2, C4, C6, TTP7h, CCP5h, CCP3h, CCP1h, CCP2h, CCP4h, CCP6h, TTP8h, CP5, CP3, CP1, CPz, CP2, CP4, CP6, CPP5h, CPP3h, CPP1h, CPP2h, CPP4h, CPP6h, P3, P1, Pz, P2, P4, PPO1h, PPO2h, eog-l, eog-m, eog-r, thumb_near, thumb_far, thumb_index, index_near, index_far, index_middle, middle_near, middle_far, middle_ring, ring_near, ring_far, ring_little, litte_near, litte_far, thumb_palm, wrist_bend, roll, pitch, gesture, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy
Creating raw.info structur

/opt/conda/lib/python3.10/contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'armeodummy'}. Applying running numbers for duplicates.
  next(self.gen)


Extracting EDF parameters from /kaggle/input/bnci2020/S01_MI/motorimagination_subject1_run2.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
F3, F1, Fz, F2, F4, FFC5h, FFC3h, FFC1h, FFC2h, FFC4h, FFC6h, FC5, FC3, FC1, FCz, FC2, FC4, FC6, FTT7h, FCC5h, FCC3h, FCC1h, FCC2h, FCC4h, FCC6h, FTT8h, C5, C3, C1, Cz, C2, C4, C6, TTP7h, CCP5h, CCP3h, CCP1h, CCP2h, CCP4h, CCP6h, TTP8h, CP5, CP3, CP1, CPz, CP2, CP4, CP6, CPP5h, CPP3h, CPP1h, CPP2h, CPP4h, CPP6h, P3, P1, Pz, P2, P4, PPO1h, PPO2h, eog-l, eog-m, eog-r, thumb_near, thumb_far, thumb_index, index_near, index_far, index_middle, middle_near, middle_far, middle_ring, ring_near, ring_far, ring_little, litte_near, litte_far, thumb_palm, wrist_bend, roll, pitch, gesture, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy
Creating raw.info structur

/opt/conda/lib/python3.10/contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'armeodummy'}. Applying running numbers for duplicates.
  next(self.gen)


Extracting EDF parameters from /kaggle/input/bnci2020/S01_MI/motorimagination_subject1_run3.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
F3, F1, Fz, F2, F4, FFC5h, FFC3h, FFC1h, FFC2h, FFC4h, FFC6h, FC5, FC3, FC1, FCz, FC2, FC4, FC6, FTT7h, FCC5h, FCC3h, FCC1h, FCC2h, FCC4h, FCC6h, FTT8h, C5, C3, C1, Cz, C2, C4, C6, TTP7h, CCP5h, CCP3h, CCP1h, CCP2h, CCP4h, CCP6h, TTP8h, CP5, CP3, CP1, CPz, CP2, CP4, CP6, CPP5h, CPP3h, CPP1h, CPP2h, CPP4h, CPP6h, P3, P1, Pz, P2, P4, PPO1h, PPO2h, eog-l, eog-m, eog-r, thumb_near, thumb_far, thumb_index, index_near, index_far, index_middle, middle_near, middle_far, middle_ring, ring_near, ring_far, ring_little, litte_near, litte_far, thumb_palm, wrist_bend, roll, pitch, gesture, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy, armeodummy
Creating raw.info structur

...

Used Annotations descriptions: ['1536', '1537', '1538', '1539', '1540', '1541', '1542']
Not setting metadata
420 matching events found
Setting baseline interval to [-0.009765625, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 420 events and 2566 original time points ...
0 bad epochs dropped
*************************************************************
Applying butterworth filter
*************************************************************
Setting up band-pass filter from 0.5 - 70 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.50, 70.00 Hz: -6.02, -6.02 dB

*************************************************************
Applying ICA
*************************************************************
Fitting ICA to data using 64 channels (please be patient, this may take a while)


/tmp/ipykernel_30/863888976.py:55: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs)


Selecting by number: 15 components
Fitting ICA took 53.2s.
Applying ICA to Epochs instance


/tmp/ipykernel_30/863888976.py:57: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  epochs = ica.apply(epochs)


    Transforming to ICA space (15 components)
    Zeroing out 1 ICA component
    Projecting back using 64 PCA components
*************************************************************
Applying baseline correction
*************************************************************
Applying baseline correction (mode: mean)


In [32]:
np.save(f"data/MI/S01/data.npy", data)
np.save(f"data/MI/S01/labels.npy", labels)

In [33]:
import os
import json

from kaggle_secrets import UserSecretsClient

In [36]:
secrets = UserSecretsClient()

os.environ['KAGGLE_USERNAME'] = secrets.get_secret("KAGGLE_USERNAME")
os.environ['KAGGLE_KEY'] = secrets.get_secret("KAGGLE_KEY")

In [37]:
meta = dict(
    id="jaskaransingh2002/bnci-numpy",
    title="bnci-numpy-dataset",
    isPrivate=True,
    licenses=[dict(name="other")]
)

with open('/kaggle/working/data/dataset-metadata.json', 'w') as f:
    json.dump(meta, f)

In [38]:
!kaggle datasets create -p "/kaggle/working/data" --dir-mode zip

Starting upload for file MI.zip
100%|██████████████████████████████████████| 7.43G/7.43G [04:03<00:00, 32.8MB/s]
Upload successful: MI.zip (7GB)
Starting upload for file ME.zip
Request failed: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Will retry in 0.8 seconds
100%|██████████████████████████████████████| 7.43G/7.43G [04:04<00:00, 32.6MB/s]
Upload successful: ME.zip (7GB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/jaskaransingh2002/bnci-numpy
